In [36]:
import torch
import pandas as pd
import numpy as np
import re
import gensim
import collections

# Data
### Helping Functions

In [68]:
def normalization(t):
    t = araby.strip_tashkeel(t)
    t = araby.normalize_hamza(t)
    t = araby.normalize_alef(t)
    t = araby.strip_tatweel(t)
    t = araby.normalize_teh(t)
    return t

### Data

In [38]:
data= np.load('../translation project/AD_NMT-master/LAV-MSA-2-both.pkl',allow_pickle=True)

In [39]:
data[0] # lav , msa

['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش',
 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة']

In [40]:
# extract only msa text
msa=[]
for i,ex in enumerate(data):
    msa_text = normalization(ex[1])
    data[i][1] = msa_text
    msa.append(msa_text)

In [41]:
msa = ' '.join(msa)

Dictionaries

In [42]:
msa_d=collections.Counter(msa.split())

In [43]:
min_count = 2

In [44]:
idx2msa = np.array([word for word,freq in msa_d.items() if freq > min_count ])

In [45]:
msa2idx = {word:i for i,word in enumerate(idx2msa)}

In [46]:
msa_data = [' '.join([i for i in t[1].split() if (msa2idx.get(i,-1) != -1 and t[1] != '')]) for t in data]

In [47]:
msa_data = [i for i in msa_data if i != '']

Load Embeddings

In [48]:
t_model = gensim.models.Word2Vec.load('../resources/models/word vectors/word2vec/wiki/full_grams_cbow_100_wiki/full_grams_cbow_100_wiki.mdl')

['.',
 'في',
 'من',
 'علي',
 '(',
 ')',
 'الي',
 'ان',
 'و',
 'عام',
 ':',
 'التي',
 'او',
 ',',
 'كان',
 'مع',
 'هو',
 'عن',
 'هذه',
 'الذي',
 'هي',
 'حيث',
 'بعد',
 'هذا',
 'كما',
 'وقد',
 'ذلك',
 'ما',
 'سنه',
 'بين',
 'كانت',
 'وفي',
 'م',
 'وهو',
 'قبل',
 'لا',
 'قد',
 'ثم',
 'كل',
 'حتي',
 'انه',
 'مثل',
 'خلال',
 'ايضا',
 'وهي',
 'وكان',
 'تم',
 'له',
 'اي',
 'بعض',
 'مدينه',
 'فيها',
 'اكثر',
 'الا',
 'ومن',
 'والتي',
 'يمكن',
 'ولكن',
 'سكانها',
 'تقع',
 'العديد',
 'نسمه',
 'منها',
 'غير',
 'بها',
 'ه',
 'يكون',
 'المدينه',
 'اول',
 'لها',
 'اما',
 'بن',
 'هناك',
 'عند',
 'عندما',
 'احد',
 'عدد',
 'تلك',
 'لم',
 'يتم',
 'فقد',
 'تكون',
 'فان',
 'انها',
 'اخري',
 'فيه',
 'منذ',
 'وذلك',
 '–',
 'ال',
 '%',
 'مجموعه',
 'ب',
 'ولد',
 'به',
 'الاول',
 'لكن',
 'حوالي',
 'بسبب',
 'وكانت',
 'عليه',
 'والذي',
 'نسمه_حسب',
 'منطقه',
 'العام',
 'محمد',
 'فقط',
 'الاولي',
 'المنطقه',
 'الاخري',
 'عده',
 'اخر',
 'اذا',
 'بلغ_تعداد',
 'يقدر_عدد',
 'سكانها_ب',
 'قام',
 'العمل',
 'قال',
 'موا

In [70]:
for i in i2l[::-1]:
    if t_model.wv.key_to_index[i]:
        print(i)

ي
ه
ج
خ
ت
ف
ن
ب
ظ
ل
س
و
ح
ذ
ك
ع
ض
ص
ر
غ
ث
ش
ا
ق
ء
ز
م
ط
د


In [69]:
i2l = list(set(normalization(araby.LETTERS)))
i2v = {}
for letter in i2l:
    if letter in t_model.wv.index_to_key :
        i2v[i] = t_model.wv.get_vector(letter)
    
i2l = i2l +  ' ' + '*'
l2i = {v:i for i,v in enumerate(i2l)}

TypeError: can only concatenate list (not "str") to list

Deep Learning

In [25]:
from torch.utils.data import Dataset,DataLoader

In [26]:
from torch.nn.utils.rnn import pad_sequence 

In [27]:
def noise(txt):
    print('txt',txt)
    new_i2l = i2l + [''] 
    noise_sz = np.random.randint(0,len(txt),1)
    replace_idx = np.random.choice(len(txt),noise_sz,replace=False)
    letters_idx = np.random.choice(len(new_i2l),noise_sz,replace=True)
    txt = list(txt)
    for rep,let in zip(replace_idx,letters_idx):
        txt[rep] = new_i2l[let]
    return ''.join(txt)

In [29]:
class arrDs(Dataset):
    def __init__(self,txt_list,l2i):
        self.data = txt_list
        self.l2i = l2i
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        X = noise(self.data[idx])
        Y = self.data[idx]
        
        X = torch.tensor([self.l2i[i] for i in X])
        Y = torch.tensor([self.l2i[i] for i in Y])
        #numerilize
        return (X,Y)

In [30]:
ds = arrDs(msa_data,l2i)

In [31]:
ds[0]

txt لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و فيها


KeyError: 'ى'

In [147]:
def collate_fn(data):
    data,label  = data
    #data = [torch.tensor(i).clone().detach() for i in data]
    data = pad_sequence(data,batch_first=True)
    return data

In [148]:
dl = DataLoader(ds,batch_size=2,collate_fn=collate_fn)

In [149]:
for i in dl:
    pass

txt اعرف واحده ذهبت فرنسا غرفه وضعت
txt اذهب تقدم يسارا
txt يجب يكون موضوع
txt تبتعد تفقد الطريق
txt اقصد صراحه ياامي ايضا كرهته يوم حضرته مثلما خالتي كرهته
txt قال موضوعك جيد ساقول كنت
txt كنت اردت اخوتي اشتري الطيبه اخر الليل متاكده تصبح
txt ايضا عندما بدوا المنزل مرضت لماذا
txt كلمات جميله كليب جميل ويحتوي تشجيع لبس الحجاب الحقيقه تعجبت الكلام الغريب تحدثتي الاغنيه
txt العرض الامر وظيفه محترمه تعني محل للسكن وسياره
txt يخاف الحرب يخاف شي مهم يخسره
txt الجزاير العاصمه
txt الخاصه قليله
txt المشكله
txt اعملي جيدا
txt جيت تعرف يجب تفعله تحسبين العمل سهل بلادنا
txt السلامه
txt اكتب حاليا
txt اسمعها
txt قالت مراد قال سامي
txt يقول تفعله قال وجدتك تحاول تفكها ففعلت بدلا
txt المهم يكون
txt قالت عايده المنزل
txt اخذت الدوله
txt اظن سليمه
txt فعلنا
txt بدات الحلقه
txt تم ترسيمها
txt المسكينه عانت كثيرا
txt اداره
txt مازالت البلد يتعب ويشقي الشباب
txt اخترت فيفري المقايس
txt المشكله تكمن النظام باكمله
txt شيء


KeyError: 'ء'